<a href="https://colab.research.google.com/github/lucas6028/machine_learning/blob/main/CNN/sports_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive

import os
import numpy as np
import matplotlib.pyplot as plt
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/sports-classification")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9


## Use TensorFlow ImageDataGenerator for Image Loading and Augmentation

Use ImageDataGenerator to preprocess the images (e.g., rescale, rotate, flip) and create generators for training and validation data:

In [ ]:
train_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/train'
test_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/test'
val_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/valid'

# Create ImageDataGenerator instances for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

validation_datagen = ImageDataGenerator(rescale=1./255,)

# Flow training images in batches from the dataset directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' if you have multiple classes
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' if you have multiple classes
)

Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


## Define the CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer with the number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 100)                 │          51,300 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,447,396 (169.55 MB)

 Trainable params: 44,447,396 (169.55 MB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

Use early stopping to avoid overfitting.

In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


422/422 ━━━━━━━━━━━━━━━━━━━━ 180s 401ms/step - accuracy: 0.0563 - loss: 4.3813 - val_accuracy: 0.2220 - val_loss: 3.5299
Epoch 2/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 157s 364ms/step - accuracy: 0.2786 - loss: 2.9051 - val_accuracy: 0.3940 - val_loss: 2.4097
Epoch 3/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 160s 372ms/step - accuracy: 0.4323 - loss: 2.2161 - val_accuracy: 0.4420 - val_loss: 2.2234
Epoch 4/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 157s 368ms/step - accuracy: 0.5270 - loss: 1.8048 - val_accuracy: 0.4720 - val_loss: 2.0579
Epoch 5/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 160s 374ms/step - accuracy: 0.5920 - loss: 1.5024 - val_accuracy: 0.4720 - val_loss: 2.0999
Epoch 6/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 202s 375ms/step - accuracy: 0.6663 - loss: 1.2307 - val_accuracy: 0.4580 - val_loss: 2.3064
Epoch 7/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 202s 374ms/step - accuracy: 0.7040 - loss: 1.0709 - val_accuracy: 0.4880 - val_loss: 2.0652
Epoch 8/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 162s 379ms/step - accuracy: 0.7601 - loss: 0.87

## Evaluate the model

Once trained, you can evaluate the model on a test set or use it for predictions.

In [ ]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.5239 - loss: 1.9251
Validation Loss: 1.9574838876724243, Validation Accuracy: 0.5320000052452087


## Store the model

In [ ]:
drive.mount('/content/drive')

# Save TensorFlow model to Google Drive
model.save('/content/drive/MyDrive/model.h5')

Mounted at /content/drive
